## This notebook is for the Clustering and Segmentation Assignment

In [62]:
# download libraries
import pandas as pd
import numpy as np
import requests
import urllib.request
import time
!pip install beautifulsoup4
!pip install lxml
!pip install et_xmlfile
from bs4 import BeautifulSoup
from urllib.request import urlopen
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#soup = BeautifulSoup(html, 'html.parser')
#tables = soup.find_all('table')


In [63]:
#html = urlopen(url) 
response=requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


### Find the relevant table

In [64]:
all_tables=soup.find_all("table")
table1=soup.find("table", class_='wikitable sortable')


In [65]:
len(table1.find_all('tr'))


181

### Create lists for different table rows

In [66]:
A=[]
B=[]
C=[]

for row in table1.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].text.strip())
        B.append(cells[1].text.strip())
        C.append(cells[2].text.strip())
        

### Create Dataframe with the lists above

In [73]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Drop rows with Borough='Not assigned'

In [68]:
indexNames = df[ (df['Borough'] == 'Not assigned')].index
indexNames

Int64Index([  0,   1,   7,  10,  15,  16,  19,  24,  25,  28,  29,  33,  34,
             35,  37,  38,  42,  43,  44,  51,  52,  53,  60,  61,  62,  69,
             70,  71,  78,  79,  87,  88,  96,  97, 101, 105, 106, 110, 115,
            118, 119, 123, 124, 125, 127, 128, 131, 132, 133, 134, 136, 137,
            140, 141, 145, 146, 149, 150, 154, 155, 158, 159, 161, 162, 163,
            164, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 179],
           dtype='int64')

In [69]:
df.drop(indexNames , inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Group the dataframe by field 'Postal Code'

In [70]:
df['Postal Code'].groupby([df.Borough, df.Neighborhood]).apply(list).reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [71]:
indexNames1 = df[ (df['Neighborhood'] == 'Not assigned')].index
indexNames1

Int64Index([], dtype='int64')

In [72]:
df.shape

(103, 3)

### Create the GeoCode dataframe with csv file at location http://cocl.us/Geospatial_data

In [59]:
Geospatial_data="http://cocl.us/Geospatial_data"
GeoCode=pd.read_csv(Geospatial_data)
GeoCode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [48]:
GeoCode.shape

(103, 3)

### Merge the two dataframes: GeoCode and df to get the new dataframe with all the details at one place

In [58]:

df_new=df.merge(GeoCode, how='left')
df_new.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
